In [1]:
import pandas as pd
import numpy as np
import os
from cassandra.cluster import Cluster
from datetime import datetime
import plotly.graph_objects as go
import settrade_v2
from settrade_v2 import Investor
import time

In [2]:
cluster = Cluster(['127.0.0.1']) 
session = cluster.connect()

In [7]:
rows = session.execute("SELECT keyspace_name FROM system_schema.keyspaces;")
print("📂 Keyspaces:")
for row in rows:
    print("-", row.keyspace_name)

📂 Keyspaces:
- system_auth
- data_stock
- system_schema
- system_distributed
- my_keyspace
- system
- info_stock
- settrade_api
- system_traces
- data_quran


In [8]:
keyspace = "data_stock"   # 👈 แก้เป็น keyspace ของคุณ
rows = session.execute(f"SELECT table_name FROM system_schema.tables WHERE keyspace_name='{keyspace}';")
print(f"\n📑 Tables in keyspace '{keyspace}':")
for row in rows:
    print("-", row.table_name)



📑 Tables in keyspace 'data_stock':
- candlestick_data
- smartset_finan_data


In [9]:
table = "candlestick_data"
rows = session.execute(f"SELECT column_name, kind, type FROM system_schema.columns WHERE keyspace_name='{keyspace}' AND table_name='{table}';")
print(f"\n📊 Columns in {keyspace}.{table}:")
for row in rows:
    print(f"- {row.column_name} ({row.kind}): {row.type}")


📊 Columns in data_stock.candlestick_data:
- close_price (regular): float
- high_price (regular): float
- low_price (regular): float
- open_price (regular): float
- symbol (partition_key): text
- time (clustering): timestamp
- value (regular): float
- volume (regular): bigint


In [10]:
table = "smartset_finan_data"
rows = session.execute(f"SELECT column_name, kind, type FROM system_schema.columns WHERE keyspace_name='{keyspace}' AND table_name='{table}';")
print(f"\n📊 Columns in {keyspace}.{table}:")
for row in rows:
    print(f"- {row.column_name} ({row.kind}): {row.type}")


📊 Columns in data_stock.smartset_finan_data:
- adjustedpriceflag (regular): text
- aomvalue (regular): double
- aomvolume (regular): double
- average (regular): decimal
- bvps (regular): double
- close (regular): decimal
- date (clustering): date
- dividendyield (regular): double
- high (regular): decimal
- low (regular): decimal
- marketcap (regular): double
- open (regular): decimal
- pbv (regular): double
- pe (regular): double
- prior (regular): decimal
- securitytype (regular): text
- symbol (partition_key): text
- totalvalue (regular): double
- totalvolume (regular): double
- trvalue (regular): double
- trvolume (regular): double
- volumeturnover (regular): double


In [11]:
session.set_keyspace('data_stock')

EDA TECHNICAL

In [53]:
rows = session.execute("""
    SELECT * FROM candlestick_data WHERE symbol = 'AOT' ALLOW FILTERING;
""")

In [54]:
df = pd.DataFrame(rows, columns=[
    'symbol', 'time', 'open_price', 'high_price', 'low_price', 'close_price', 'volume', 'value'
])

# ✅ แปลงชนิดเวลา + เรียงลำดับ
df['time'] = pd.to_datetime(df['time'])
df = df.sort_values('time').reset_index(drop=True)


In [55]:
df.head()

,symbol,time,open_price,high_price,low_price,close_price,volume,value
0,AOT,2022-06-13,68.25,68.75,68.00,68.25,0.0,23102580
1,AOT,2022-06-14,68.00,68.50,67.25,68.25,0.0,23044390
2,AOT,2022-06-15,68.25,68.25,67.50,67.75,0.0,17456121
3,AOT,2022-06-16,67.50,69.00,67.50,68.75,0.0,34740306
4,AOT,2022-06-17,67.75,68.50,67.25,67.25,0.0,34111893


In [56]:
import plotly.graph_objects as go

# ✅ สร้างกราฟ Candlestick
fig = go.Figure()

# 🔹 แท่งเทียนราคา
fig.add_trace(go.Candlestick(
    x=df['time'],
    open=df['open_price'],
    high=df['high_price'],
    low=df['low_price'],
    close=df['close_price'],
    
))

# 🔹 Volume bar (แถบปริมาณการซื้อขาย)
fig.add_trace(go.Bar(
    x=df['time'],
    y=df['volume'],
    marker_color='rgba(128,128,128,0.3)',
    yaxis='y2',
    
))

# ✅ ปรับ layout ให้เหมือน TradingView
fig.update_layout(
    
    xaxis_rangeslider_visible=False,
    yaxis=dict(title='Price'),
    yaxis2=dict(
        title='Volume',
        overlaying='y',
        side='right',
        showgrid=False
    ),
    height=600,
    template='plotly_white'
)

fig.show()


In [57]:
import plotly.graph_objects as go

# ✨ คำนวณ EMA
for span in [5, 15, 35, 89, 200]:
    df[f'EMA{span}'] = df['close_price'].ewm(span=span, adjust=False).mean()

# ✅ สร้างกราฟ Candlestick
fig = go.Figure()

# 🔹 แท่งเทียนราคา
fig.add_trace(go.Candlestick(
    x=df['time'],
    open=df['open_price'],
    high=df['high_price'],
    low=df['low_price'],
    close=df['close_price'],
    name="Candlestick"
))

# 🔹 Volume bar (แถบปริมาณการซื้อขาย)
fig.add_trace(go.Bar(
    x=df['time'],
    y=df['volume'],
    marker_color='rgba(128,128,128,0.3)',
    yaxis='y2',
    name="Volume"
))

# 🔹 เพิ่ม EMA หลายเส้น
colors = {5:"blue", 15:"orange", 35:"green", 89:"red", 200:"purple"}
for span, color in colors.items():
    fig.add_trace(go.Scatter(
        x=df['time'],
        y=df[f'EMA{span}'],
        mode='lines',
        name=f'EMA {span}',
        line=dict(color=color, width=1.5)
    ))

# ✅ ปรับ layout ให้เหมือน TradingView
fig.update_layout(
    title="Candlestick + EMA (5, 15, 35, 89, 200)",
    xaxis_rangeslider_visible=False,
    yaxis=dict(title='Price'),
    yaxis2=dict(
        title='Volume',
        overlaying='y',
        side='right',
        showgrid=False
    ),
    height=600,
    template='plotly_white'
)

fig.show()


EDA FINANCE DATA

In [59]:
import pandas as pd

def fetch_symbol_df(session, symbol: str, table: str = "smartset_finan_data", keyspace: str = None):
    """
    ดึงข้อมูลจาก Cassandra table แล้วแปลงเป็น Pandas DataFrame
    
    Args:
        session : cassandra-driver Session
        symbol  : สัญลักษณ์หุ้น เช่น "AOT"
        table   : ชื่อตาราง (default = smartset_finan_data)
        keyspace: ชื่อ keyspace (optional ถ้า session.set_keyspace() เรียบร้อยแล้ว)
    
    Returns:
        df : DataFrame พร้อมคอลัมน์จัดเรียงและ dtype แปลงแล้ว
    """
    # สร้าง query
    ks_prefix = f"{keyspace}." if keyspace else ""
    query = f"SELECT * FROM {ks_prefix}{table} WHERE symbol='{symbol}';"
    
    rs = session.execute(query)
    rows_list = [r._asdict() for r in rs]
    if not rows_list:
        return pd.DataFrame()  # ว่าง
    
    df = pd.DataFrame(rows_list)

    # จัดลำดับคอลัมน์ตาม schema
    cols = [
        "symbol","date","adjustedpriceflag","aomvalue","aomvolume","average","bvps","close",
        "dividendyield","high","low","marketcap","open","pbv","pe","prior","securitytype",
        "totalvalue","totalvolume","trvalue","trvolume","volumeturnover"
    ]
    df = df.reindex(columns=cols)

    # แปลง dtype
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    num_cols = ["aomvalue","aomvolume","average","bvps","close","dividendyield","high","low",
                "marketcap","open","pbv","pe","prior","totalvalue","totalvolume",
                "trvalue","trvolume","volumeturnover"]
    df[num_cols] = df[num_cols].apply(pd.to_numeric, errors="coerce")
    
    return df.sort_values("date").reset_index(drop=True)


In [60]:
df = fetch_symbol_df(session, "AOT", keyspace="data_stock")
print(df.head())


  symbol date adjustedpriceflag     aomvalue   aomvolume    average  bvps  \
0    AOT  NaT                 N  578963650.0  14492200.0  39.950018  8.62   

   close  dividendyield  high  ...   open   pbv     pe  prior  securitytype  \
0  39.75           1.99  40.5  ...  40.25  4.61  29.53   40.5            CS   

    totalvalue totalvolume  trvalue  trvolume  volumeturnover  
0  579141926.0  14496658.0   3975.0     100.0        0.101477  

[1 rows x 22 columns]
